In [1]:
import numpy as np
import json

In [2]:
file = "/home/sue/Projects/tensoul/output/sue.json"

with open(file, "r") as f:
    haifu = json.load(f)

In [3]:
# //   11-19    - 1-9 man
# //   21-29    - 1-9 pin
# //   31-39    - 1-9 sou
# //   41-47    - ESWN WGR
# //   51,52,53 - aka 5 man, pin, sou
def num2hai(num):
    if num < 40:
        f = ["m", "p", "s"]
        return f"{num%10}{f[num//10 - 1]}"
    if num < 50:
        f = "東南西北白発中"
        return f[num%10 - 1]
    if num <60:
        f = ["5m", "5p", "5s"]
        return f[num%10 - 1]
    if num == 60:
        return "D"
def convert(num):
    if isinstance(num, int):
        return num2hai(num), None
    if "p" in num:
        splited = num.split("p")[1]
        return num2hai(int(splited[:2])), "N"
    if "c" in num:
        splited = num.split("c")[1]
        return num2hai(int(splited[:2])), "C"+num2hai(int(splited[2:4]))+num2hai(int(splited[4:6]))
    if "k" in num:
        splited = num.split("k")[1]
        return num2hai(int(splited[:2])), "addK"
    if "m" in num:
        splited = num.split("m")[1]
        return num2hai(int(splited[:2])), "K"
    if "a" in num:
        splited = num.split("a")[1]
        return num2hai(int(splited[:2])), "anK"
    if "r" in num:
        return num2hai(int(num[1:])), "R"

def convert_list(l):
    return "".join([num2hai(c) for c in l])


In [4]:
def title(line):
    first = ["東1局", "東2局", "東3局", "東4局", "南1局", "南2局", "南3局", "南4局"][line[0]]
    second = f"{line[1]}本場"
    return first + " " + second
def info(line):
    return line[0]

def checklen(log, indices, moda=0):
    for i in range(4):
        if indices[i] < len(log[3 * i + 5+moda]):
            return True
    return False


log = haifu[63]["log"][0]

def to_format(log):
    kaze = ["東", "北", "西","南" ]

    lines = [title(log[0]), info(log[-1])]
    lines += [
        f"[1{kaze[log[0][0]%4]}]"+convert_list(sorted(log[4][:14])), 
        f"[2{kaze[(log[0][0]-1)%4]}]"+convert_list(sorted(log[7][:14])), 
        f"[3{kaze[(log[0][0]-2)%4]}]"+convert_list(sorted(log[10][:14])), 
        f"[4{kaze[(log[0][0]-3)%4]}]"+convert_list(sorted(log[13][:14])), 
        f"[表ドラ]{convert_list(log[2])} [裏ドラ]{convert_list(log[3])}",
        ]

    curr = log[0][0] % 4
    inds_mo = [0, 0, 0, 0]
    inds_da = [0, 0, 0, 0]
    mo = lambda n, inds: convert(log[3 * n + 5][inds[n]])
    da = lambda n, inds: convert(log[3 * n + 6][inds[n]])
    nt = lambda curr: (curr + 1) % 4
    actions = []
    curr_da = None
    p = lambda curr : curr + 1
    while checklen(log, inds_mo):
        # peng kang chi mo 
        if inds_mo[curr] < len(log[3 * curr + 5]):
            curr_mo, a = mo(curr, inds_mo)
            if a == None:
                a = "G"
            elif a[0] == "C" and curr_mo == curr_da:
                # check chi
                curr_mo = ""
        else:
            curr_mo, a = None, None
        # check peng kang
        for i in range(3):
            test = (curr + i)%4
            if inds_mo[test] >= len(log[3 * test + 5]):
                continue
            test_mo, test_a = mo(test, inds_mo)
            if test_a=="N" and test_mo == curr_da:
                curr = test
                a = test_a
                curr_mo = ""
                break
            elif test_a=="K" and test_mo == curr_da:
                curr = test
                a = test_a
                curr_mo = test_mo
    
        if curr_mo == None:
            return "error"
        actions.append(f"{p(curr)}{a}{curr_mo}")
        inds_mo[curr] += 1
        if a == "K":
            inds_da[curr] += 1
            continue

        if inds_da[curr] >= len(log[3 * curr + 6]):
            break
        
        curr_da, r = da(curr, inds_da)

        if r == "R":
            actions.append(f"{curr+1}R")
        if r == None or r == "R":
            if curr_da == "D" or curr_da==curr_mo:
                d = "D"
                curr_da = curr_mo
            else:
                d = "d"
        elif "K" in r:
            d = "K"
    
        actions.append(f"{curr+1}{d}{curr_da}")
        inds_da[curr] += 1
        if d != "K":
            curr = nt(curr)
    for n in range(0, len(actions), 15):
        lines.append("* " + " ".join(actions[n: min(len(actions), n+15)]))

    return lines
f = open("/home/sue/Projects/tenpai_fork/data/xsx.txt", "w")
for taisen in haifu:
    if taisen["ratingc"]=="PF3":
        continue
    for log in taisen["log"]:
        res = to_format(log)
        if res != "error":
            f.write("\n".join(to_format(log)))
            f.write("\n\n")
f.close()

In [124]:
haifu[0]["log"][0]

[[0, 0, 0],
 [25000, 25000, 25000, 25000],
 [47],
 [45],
 [34, 42, 21, 11, 28, 27, 16, 11, 39, 43, 53, 31, 13],
 [17, 27, 17, 11, 25, 24, 18, 31, 12, 23, 47, 44, 22, 26, 39],
 [42, 39, 21, 31, 60, 60, 17, 13, 43, 60, 60, 31, 60, 12, 60],
 [33, 38, 22, 27, 42, 19, 17, 12, 33, 34, 15, 46, 37],
 [21, 17, 31, 27, 37, 25, 'p171717', '3333p33', 41, 42, 22, 46, 39, 37, 12],
 [42, 46, 60, 21, 22, 12, 19, 15, 38, 60, 41, 60, 22, 25, 60],
 [47, 39, 14, 12, 13, 11, 38, 42, 43, 34, 33, 19, 28],
 [16, 26, 41, 15, 33, 14, 35, 51, 16, 45, 45, 18, 23, 32, 36],
 [42, 47, 60, 43, 28, 26, 33, 19, 'r38', 60, 60, 60, 60, 60, 60],
 [13, 16, 36, 45, 32, 29, 25, 37, 44, 36, 43, 19, 23],
 [26, 18, 44, 38, 35, 13, 22, 19, 28, 29, 18, 21, 14, 28],
 [29, 43, 23, 32, 19, 18, 60, 60, 38, 45, 29, 60, 18, 36],
 ['和了',
  [-5200, 0, 6200, 0],
  [2, 0, 2, '40符3飜5200点', '一盃口(1飜)', '立直(1飜)', '赤ドラ(1飜)', '裏ドラ(0飜)']]]